In [ ]:
import os 
import groq 
from dotenv import find_dotenv, load_dotenv
from langchain_groq import ChatGroq
load_dotenv(find_dotenv())
groq.api_key = os.getenv("GROQ_API_KEY")

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm = ChatGroq(model="qwen/qwen3-32b")
llm_with_tools = llm.bind_tools([multiply])

c:\Users\61068067\python_langgraph\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# use the built-in ToolNode and simply pass a list of tools to initialize it
# use the built-in tools_condition as the conditional edge
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)
graph = builder.compile()

# View
#display(Image(graph.get_graph().draw_mermaid_png()))

In [4]:
from langchain_core.messages import HumanMessage
#messages = [HumanMessage(content="Hello, what is 2 multiplied by 2?")]
messages = [HumanMessage(content="Hello What is Generative AI")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello What is Generative AI
================================== Ai Message ==================================

Generative AI is a type of artificial intelligence that can create new content, such as text, images, music, or even code, based on patterns it has learned from vast amounts of training data. It uses techniques like deep learning to understand the structure and style of existing examples and then generates original outputs that mimic those patterns. For instance, models like GPT (Generative Pre-trained Transformer) can write stories, answer questions, or draft emails, while others like DALL-E can create images from textual descriptions. Its applications range from creative writing to design, gaming, and more. How can I assist you further? 😊
